<a href="https://colab.research.google.com/github/likithachinnibunny/Zeotap_DataScience_Intern_Tasks/blob/main/Sompalli_Likhitha_Lookalike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

customers = pd.read_csv("https://drive.google.com/uc?id=1bu_--mo79VdUG9oin4ybfFGRUSXAe-WE")
products = pd.read_csv("https://drive.google.com/uc?id=1IKuDizVapw-hyktwfpoAoaGtHtTNHfd0")
transactions = pd.read_csv("https://drive.google.com/uc?id=1saEqdbBB-vuk2hxoAf4TzDEsykdKlzbF")

transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

customer_features = merged_data.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'Quantity': 'sum',
    'Price': 'mean',
    'Category': lambda x: x.mode()[0] if not x.empty else None
}).reset_index()

customer_features = pd.get_dummies(customer_features, columns=['Category'], dummy_na=False)

scaler = StandardScaler()
numeric_features = customer_features.drop(columns=['CustomerID'])
scaled_features = scaler.fit_transform(numeric_features)

similarity_matrix = cosine_similarity(scaled_features)

similarity_df = pd.DataFrame(similarity_matrix, index=customer_features['CustomerID'], columns=customer_features['CustomerID'])

lookalike_results = {}
for cust_id in customer_features['CustomerID'][:20]:
    similar_customers = similarity_df[cust_id].sort_values(ascending=False)[1:4]
    lookalike_results[cust_id] = list(similar_customers.items())

lookalike_df = pd.DataFrame([{"CustomerID": k, "Recommendations": v} for k, v in lookalike_results.items()])
lookalike_df.to_csv("FirstName_LastName_Lookalike.csv", index=False)

print("Lookalike model results saved successfully!")
